In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
from scipy.signal import welch
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
datapath = os.path.join(os.getcwd(), '..', 'datasets', 'co_feats32.pkl')

with open(datapath, 'rb') as f:
    data = pickle.load(f)

In [ ]:
"prototype, not really based in any sound logic just wanted to throw something at the wall"

In [ ]:
import os
import pickle
import numpy as np
from scipy.signal import welch

def load_dataset(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".pkl":
        with open(path, "rb") as f:
            return pickle.load(f)
    elif ext == ".npz":
        return dict(np.load(path, allow_pickle=True))
    else:
        raise ValueError(f"Unsupported file: {path}")

# look up spiking freq in actual paper to fill in 
def compute_sbp(neural, fs=1000 / 3.5, band=(300, 6000)):
    freqs, psd = welch(neural, fs=fs, nperseg=min(neural.shape[0], int(fs)), axis=0)
    band_mask = (freqs >= band[0]) & (freqs <= band[1])
    sbp = np.trapz(psd[band_mask, :], freqs[band_mask], axis=0)
    return sbp
    
def rank_datasets_by_mean_sbp(folder, fs=30000, band=(300, 6000), top_k_channels=10):
    results = []
    for fname in sorted(os.listdir(folder)):
        if not fname.lower().endswith((".pkl", ".npz")):
            continue
        path = os.path.join(folder, fname)
        try:
            data = load_dataset(path)
            if "neural" not in data:
                continue
            neural = np.asarray(data["neural"])
            sbp = compute_sbp(neural, fs=fs, band=band)
            mean_sbp = float(np.mean(sbp))
            top_channels = np.argsort(sbp)[::-1][:top_k_channels]
            results.append({
                "path": path,
                "mean_sbp": mean_sbp,
                "top_channels_idx": top_channels.tolist(),
            })
        except Exception as e:
            print(f"[WARN] Skipping {path}: {e}")
    results.sort(key=lambda x: x["mean_sbp"], reverse=True)
    return results

def summarize(results, max_rows=10):
    print(f"\nDatasets ranked by mean SBP (top {min(max_rows, len(results))}):\n")
    for i, r in enumerate(results[:max_rows], 1):
        print(f"{i:2d}. {os.path.basename(r['path'])} | mean SBP: {r['mean_sbp']:.3e} | top channels: {r['top_channels_idx'][:5]}...")

if __name__ == "__main__":
    folder = "/home/efri-student22/hackathon-BMI/datasets"
    results = rank_datasets_by_mean_sbp(folder, fs=30000, band=(300, 1000))
    summarize(results)

    if results:
        best = results[0]
        print("\nSBP datasets by mean:")
        print(" Mean SBP:", best["mean_sbp"])
        print(" Top channel:", best["top_channels_idx"])


In [ ]:
"Channel exclusion, this was done using our comparison to mean velocity and (explain with disertation in notes), working rand pile {58, 94, 31, 65,}"

In [ ]:
# adding in exclusion
exclude = {31, 11, 69, 1, 62, 94, 65, 58, 51, 30}
C_all = data['neural'].shape[1]
keep_idx = [ch for ch in range(C_all) if ch not in exclude]

neu = data['neural'][:, keep_idx]
vel = data['behavior'][:, [2, 3]]

hist = 6
T, C = neu.shape

adjneu = np.zeros((T - hist, C, hist + 1))
for h in range(hist + 1):
    adjneu[:, :, h] = neu[h:T - hist + h, :]

adjneu = adjneu.reshape(T - hist, C * (hist + 1))
adjvel = vel[hist:, :]

print(f"Excluded {C_all - C} channels → kept {C}")
assert adjneu.shape[0] == adjvel.shape[0]

X_train, X_test, y_train, y_test = train_test_split(adjneu, adjvel, test_size=0.2, shuffle=False)

ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)


In [ ]:
# test the ridge regression
yhat = ridge.predict(X_test)

# show test r^2
print(f'Test R2: {ridge.score(X_test, y_test)}')

In [ ]:
# plot predicted vs. true kinematics

fig, ax = plt.subplots(1,2, figsize = (9, 4))
finger = ['IDX','MRP']
for i in range(2):
    ax[i].plot(y_test[0:100,i], c='k')
    ax[i].plot(yhat[0:100,i], c='r')
    ax[i].set(xlim=(0,100),title=f'{finger[i]} velocity (%flex/bin)')
    if i == 0:
        ax[i].legend(('True', 'Predicted'))

In [ ]:
"pearson corrilation, method to madness"

In [ ]:
# import pickle
# import os
# import pandas as pd
# pd.set_option('display.max_row', None)
# from scipy import stats
# import numpy as np
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# %matplotlib inline
# from ipywidgets import *

# from sklearn.linear_model import Ridge
# from sklearn.model_selection import train_test_split
# from sklearn.decomposition import PCA

# datapath = os.path.join(os.getcwd(), '..', 'datasets', 'co_feats32.pkl')

# with open(datapath, 'rb') as f:
#     data = pickle.load(f)

In [ ]:
# Extract neural (SBP) and velocity data
neu = data['neural']  # Nx96 array
vel = data['behavior'][:, [2, 3]]  # Nx2 array (IDX_velocity, MRP_velocity)

print(f"Neural data shape: {neu.shape}")
print(f"Velocity data shape: {vel.shape}")

# Calculate median velocity for splitting
vel_combined = np.linalg.norm(vel, axis=1)  # Combined velocity magnitude
median_vel = np.median(vel_combined)

print(f"\nMedian velocity: {median_vel:.4f}")

# Split data into high and low velocity groups
high_vel_mask = vel_combined >= median_vel
low_vel_mask = vel_combined < median_vel

# Group neural data by velocity condition
grouped = {'neural': [], 'behavior': []}
grouped['neural'].append(neu[high_vel_mask])  # High velocity group
grouped['neural'].append(neu[low_vel_mask])   # Low velocity group
grouped['behavior'].append(vel[high_vel_mask])
grouped['behavior'].append(vel[low_vel_mask])

print(f"High velocity samples: {grouped['neural'][0].shape[0]}")
print(f"Low velocity samples: {grouped['neural'][1].shape[0]}")

# Calculate mean neural activity for each group and channel
mean_neural_high = np.mean(grouped['neural'][0], axis=0)  # 96 channels
mean_neural_low = np.mean(grouped['neural'][1], axis=0)   # 96 channels

print(f"\nMean neural high shape: {mean_neural_high.shape}")
print(f"Mean neural low shape: {mean_neural_low.shape}")

# Get velocity values for each group
vel_high = vel_combined[high_vel_mask]
vel_low = vel_combined[low_vel_mask]

# Run correlations for each channel
results = []

for channel in range(neu.shape[1]):
    # Get neural data for this channel in high and low velocity conditions
    high_vel_neural = grouped['neural'][0][:, channel]
    low_vel_neural = grouped['neural'][1][:, channel]
    
    # Correlate neural activity with velocity within high velocity group
    pearson_r_high, pearson_p_high = stats.pearsonr(high_vel_neural, vel_high)
    spearman_r_high, spearman_p_high = stats.spearmanr(high_vel_neural, vel_high)
    
    # Correlate neural activity with velocity within low velocity group
    pearson_r_low, pearson_p_low = stats.pearsonr(low_vel_neural, vel_low)
    spearman_r_low, spearman_p_low = stats.spearmanr(low_vel_neural, vel_low)
    
    # Overall correlation (across both groups)
    all_neural = np.concatenate([high_vel_neural, low_vel_neural])
    all_vel = np.concatenate([vel_high, vel_low])
    pearson_r_all, pearson_p_all = stats.pearsonr(all_neural, all_vel)
    spearman_r_all, spearman_p_all = stats.spearmanr(all_neural, all_vel)
    
    results.append({
        'SBP_Channel': channel,
        'mean_high_vel': mean_neural_high[channel],
        'mean_low_vel': mean_neural_low[channel],
        'mean_diff': mean_neural_high[channel] - mean_neural_low[channel],
        'pearson_r_overall': pearson_r_all,
        'pearson_p_overall': pearson_p_all,
        'spearman_r_overall': spearman_r_all,
        'spearman_p_overall': spearman_p_all,
        'pearson_r_high': pearson_r_high,
        'pearson_p_high': pearson_p_high,
        'pearson_r_low': pearson_r_low,
        'pearson_p_low': pearson_p_low,
        'spearman_r_high': spearman_r_high,
        'spearman_p_high': spearman_p_high,
        'spearman_r_low': spearman_r_low,
        'spearman_p_low': spearman_p_low,
        'pearson_sig_005': pearson_p_all < 0.05,
        'pearson_sig_001': pearson_p_all < 0.01
    })

# Create DataFrame with results
results_df = pd.DataFrame(results)

# Add absolute correlation columns for ranking
results_df['pearson_r_overall_abs'] = np.abs(results_df['pearson_r_overall'])
results_df['spearman_r_overall_abs'] = np.abs(results_df['spearman_r_overall'])

# Display summary statistics
print("\n=== Summary Statistics ===")
print(f"Total channels tested: {len(results_df)}")
print(f"Significant overall correlation at p<0.05: {results_df['pearson_sig_005'].sum()}")
print(f"Significant overall correlation at p<0.01: {results_df['pearson_sig_001'].sum()}")

# Rank all channels by correlation magnitude (absolute value)
results_df_sorted = results_df.sort_values('pearson_r_overall_abs', ascending=False)
print("\n=== All Channels Ranked by Correlation Magnitude (Overall Pearson) ===")
print(results_df_sorted[['SBP_Channel', 'pearson_r_overall', 'pearson_r_overall_abs', 'pearson_p_overall']])

print("\n=== Analysis Complete ===")

In [ ]:
"putting it all together"

In [ ]:
exclude = channels = {
    42, 85, 12, 77,
    56, 5, 93, 38, 16,
    4, 7, 24, 40, 57,
    46, 49, 52, 61, 9,
    45, 87, 47, 22, 18,
    59, 48, 83, 88, 26,
    81, 63, 15, 35, 28,
    27, 19, 43, 55, 13,
    41, 33, 29, 95, 20,
    53, 71, 25, 92, 44,
    54, 21, 60, 76, 23,
    17, 30, 51, 58, 65,
    94, 62, 1, 69, 11,
    31
}

# fixed a bug caused by hard-coding 96 neural channels
# by dynamically reading the actual number of channels (C = neu.shape[1]), 
# code now adapts automatically, and correctly builds the decoder using only the selected channels
C_all = data['neural'].shape[1]
keep_idx = [ch for ch in range(C_all) if ch not in exclude]

neu = data['neural'][:, keep_idx]
vel = data['behavior'][:, [2, 3]]

hist = 6
T, C = neu.shape

adjneu = np.zeros((T - hist, C, hist + 1))
for h in range(hist + 1):
    adjneu[:, :, h] = neu[h:T - hist + h, :]

adjneu = adjneu.reshape(T - hist, C * (hist + 1))
adjvel = vel[hist:, :]

print(f"Excluded {C_all - C} channels → kept {C}")
assert adjneu.shape[0] == adjvel.shape[0]

X_train, X_test, y_train, y_test = train_test_split(adjneu, adjvel, test_size=0.2, shuffle=False)

ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)

In [ ]:
# test the ridge regression
yhat = ridge.predict(X_test)

# show test r^2
print(f'Test R2: {ridge.score(X_test, y_test)}')

In [ ]:
# plot predicted vs. true kinematics

fig, ax = plt.subplots(1,2, figsize = (9, 4))
finger = ['IDX','MRP']
for i in range(2):
    ax[i].plot(y_test[0:100,i], c='k')
    ax[i].plot(yhat[0:100,i], c='r')
    ax[i].set(xlim=(0,100),title=f'{finger[i]} velocity (%flex/bin)')
    if i == 0:
        ax[i].legend(('True', 'Predicted'))